In [1]:
import requests, zipfile, io


In [2]:
import geopandas as gpd

In [20]:
import pandas as pd
import json
import os
from sqlalchemy import create_engine
import sqlalchemy 

In [12]:
DBname=os.environ['DB_NAME']
postgres_psswd=os.environ['POSTGRES_PASSWORD']
postgres_user=os.environ['POSTGRES_USER']
postgres_port=str(os.environ['POSTGRES_PORT'])

In [15]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://'+postgres_user+':'+postgres_psswd+'@'+DBname+':'+postgres_port+'/superset')
# Create the connection
cnx = create_engine(postgres_str)

In [4]:
#códigos ISO y coordenadas centroides

In [16]:
url='https://raw.githubusercontent.com/DataScienceResearchPeru/covid-19_latinoamerica/master/utils/iso3166-2.csv'

In [17]:
iso=pd.read_csv(url)

In [21]:
name='iso'
schema='geo'
if not cnx.dialect.has_schema(cnx, schema):
    print('schema '+schema+' does not exist, creating it')
    cnx.execute(sqlalchemy.schema.CreateSchema(schema))
else:
    print('schema '+schema+' exists, will not be created')

schema geo does not exist, creating it


In [24]:
new_cols=[x.replace('(','').replace(')','') for x in iso.columns]

In [25]:
iso.columns=new_cols

In [26]:
iso

,Country Code,Country Name,Code,Subdivision Name Used,Subdivision Category,Number,Latitude,Longitude,Subdivision Name es,Subdivision Name qu,Subdivision Name ay,Subdivision Name ht,Subdivision Name fr,Parent Subdivision,Local Variant
0,AR,Argentina,AR-B,Buenos Aires,Province,NaN,-37.201728,-59.841070,Buenos Aires,NaN,NaN,NaN,NaN,NaN,NaN
1,AR,Argentina,AR-K,Catamarca,Province,NaN,-28.471588,-65.787721,Catamarca,NaN,NaN,NaN,NaN,NaN,NaN
2,AR,Argentina,AR-H,Chaco,Province,NaN,-27.425717,-59.024378,Chaco,NaN,NaN,NaN,NaN,NaN,NaN
3,AR,Argentina,AR-U,Chubut,Province,NaN,-43.293425,-65.111482,Chubut,NaN,NaN,NaN,NaN,NaN,NaN
4,AR,Argentina,AR-C,Ciudad Autónoma de Buenos Aires,City,NaN,-34.603684,-58.381559,Ciudad Autónoma de Buenos Aires,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,VE,Venezuela,VE-T,Trujillo,State,NaN,9.430253,-70.526493,Trujillo,NaN,NaN,NaN,NaN,NaN,NaN
405,VE,Venezuela,VE-S,Táchira,State,NaN,7.913700,-72.141613,Táchira,NaN,NaN,NaN,NaN,NaN,NaN
406,VE,Venezuela,VE-X,Vargas,State,NaN,10.589002,-66.736734,Vargas,NaN,NaN,NaN,NaN,NaN,NaN
407,VE,Venezuela,VE-U,Yaracuy,State,NaN,10.340192,-68.745306,Yaracuy,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
iso.to_sql(name, schema=schema,con=cnx,if_exists='replace')

In [7]:
#comunas
#url='https://www.bcn.cl/obtienearchivo?id=repositorio/10221/10396/2/Comunas.zip'
#r = requests.get(url)
#z = zipfile.ZipFile(io.BytesIO(r.content))
#z.extractall("/data/ETLcache/comunas_shp")

In [14]:
#provincias
#url='https://www.bcn.cl/obtienearchivo?id=repositorio/10221/10397/2/Provincias.zip'
#r = requests.get(url)
#z = zipfile.ZipFile(io.BytesIO(r.content))
#z.extractall("/data/ETLcache/provincias_shp")

In [15]:
#regiones
#url='https://www.bcn.cl/obtienearchivo?id=repositorio/10221/10398/2/Regiones.zip'
#r = requests.get(url)
#z = zipfile.ZipFile(io.BytesIO(r.content))
#z.extractall("/data/ETLcache/regiones_shp")

In [21]:
#url='https://raw.githubusercontent.com/ivanMSC/COVID19_Chile/master/utils/Mapa%20de%20Chile%20con%20Zoom%20en%20Santiago.json'
#response = requests.get(url)
#data = response.json()

In [23]:
#data

{'type': 'Topology',
 'arcs': [[[27637, 25363],
   [-142, -71],
   [-3, 78],
   [-132, -38],
   [-82, 60],
   [276, 50],
   [83, -79]],
  [[27050, 24589], [-54, -16], [7, -98], [-106, -83], [9, 244], [144, -47]],
  [[27036, 21921], [19, -63], [-76, -121], [-72, 299], [129, -115]],
  [[28236, 25693],
   [14, -65],
   [-157, 2],
   [-43, -67],
   [-30, 67],
   [-3, -82],
   [-159, 2],
   [189, 141],
   [189, 2]],
  [[28837, 24946], [-43, -64], [-349, 30], [202, 109], [190, -75]],
  [[24589, 21432],
   [-63, -11],
   [11, -100],
   [-205, 10],
   [-10, 103],
   [129, 107],
   [138, -109]],
  [[26847, 24372],
   [-219, -149],
   [-143, 71],
   [228, 245],
   [63, -108],
   [-45, -65],
   [116, 6]],
  [[28260, 24560],
   [-50, -137],
   [-176, -33],
   [-75, 74],
   [77, 119],
   [-105, 0],
   [171, 138],
   [-14, -149],
   [84, 66],
   [88, -78]],
  [[24930, 19957],
   [-44, -192],
   [58, -148],
   [-301, 185],
   [34, 64],
   [164, -5],
   [-171, 81],
   [51, 49],
   [209, -34]],
  [[243

In [34]:
#url='https://gist.githubusercontent.com/rhernandog/7d055482f5cc803852a762de873bea62/raw/2bed9aed94ab644533b5e624a4e8f165a4650d48/regiones-provincias-comunas.json'
#response = requests.get(url)
#data = response.json()

In [35]:
#gdf = gpd.GeoDataFrame(data)

In [27]:
#from mapbox import Uploader

In [28]:
#conxn = Uploader(access_token='')

In [ ]:
#resp = conxn.upload('RGB.byte.tif', 'RGB-byte-tif')